### Regional Transit System
NYC Mass Transit Spatial Layers: https://www.baruch.cuny.edu/confluence/display/geoportal/NYC+Mass+Transit+Spatial+Layers <br>
New Jersey Geographic Information Network (NJGIN) Open Data: https://njogis-newjersey.opendata.arcgis.com <br>
Also a brief introduction of transit network Page 13 https://www1.nyc.gov/assets/planning/download/pdf/planning-level/housing-economy/nyc-ins-and-out-of-commuting.pdf

<img src="regional_transit_system.png" width="600">

**The geography of the region’s transit options shapes commuters’ mode choices.** <br>
**What information (about public transportation) do we need when building the adaptive commuter model?** <br>
Commuters’ current mode choices: the only choice, or they have strong preferences even when they have multiple options (subway over bus)? <br>
If we think of commuters’ current mode choices as the results of all kinds of constraints, we only need to take their current choice, no need to model their available choices. <br>
But, the truth is that both reason exits, so if we want to add more flexibility about the impact of futrue 100% e-mobilty, no matter policy or new infrastructure, we need to get their available choices. Such as: For autos commmuters, if there will be more e-bus lines, are they willing to change to e-bus? <br>
**But for current commuters who are already using public transporation, it's hard to tell. Subway, commuter rail: already electric; ferry, bus: not yet** <br>
For now, there seems no meaning to let current public transporation commuters to choose among Subway/rail/ferry/bus

### PUMAs reachable from/to Manhattan via the regional transit system

Note: The geographical area of Residential PUMAs are different than Place of Work PUMAs, so calculate the reachable PUMAs separately.

In [1]:
import pandas as pd
pd.set_option("max_columns",0)

In [2]:
ipums_df = pd.read_csv("../../01_DataExploration_and_Engineering/disaggregated_cleaned_ipums_data.csv",index_col=0)
check_df = ipums_df[ipums_df['YEAR']==2019].reset_index(drop=True)
check_df.head(3)

,YEAR,PERWT,HOME_STATEFIP,HOME_PUMA,PUMA_NAME,SEX,AGE,HRS_WK_DAILY,TOTAL_PERSONAL_INCOME,MODE_TRANSP_TO_WORK,MODE_TRANSP_TO_WORK_HBDMATCH,HOMEOWNER_LABEL,RACE_LABEL,EDUC_LABEL,DEPARTS_FOR_WORK_HOUR,ARRIVES_AT_WORK_HOUR,COMMUTE_DIRECTION_MANHATTAN,VEHICLE_AVAILABLE,OCC_DESCRIPTION,IND_CAT,IND_DESCRIPTION,PUMAKEY_HOME,PUMAKEY_WORK,DISTANCE_KM,COGNITIVE_DIFFICULTY,AMBULATORY_DIFFICULTY,IND_LIVING_DIFFICULTY,SELFCARE_DIFFICULTY,VISION_OR_HEARING_DIFFICULTY,VISION_DIFFICULTY,HEARING_DIFFICULTY
0,2019,23.0,9,500,Litchfield County,F,46,7.4,3800,"Auto, truck, or van",AutoOccupants,Rent,White,Grades12,8,9,in,1,Personal care aides,"Educational Services, and Health Care and Soci...",Home health care services,09_00500,36_03800,170.50491,0,0,0,0,0,0,0
1,2019,48.0,9,500,Litchfield County,M,51,11.0,488000,"Auto, truck, or van",AutoOccupants,Rent,White,College_5PlusYears,8,8,in,1,"Lawyers, and judges, magistrates, and other ju...","Professional, Scientific, and Management, and ...",Legal services,09_00500,36_03800,170.50491,0,0,0,0,0,0,0
2,2019,54.0,9,500,Litchfield County,F,44,11.0,438000,"Auto, truck, or van",AutoOccupants,Rent,White,College_5PlusYears,8,8,in,1,"Lawyers, and judges, magistrates, and other ju...","Professional, Scientific, and Management, and ...",Legal services,09_00500,36_03800,170.50491,0,0,0,0,0,0,0


In [3]:
dir_in = check_df['COMMUTE_DIRECTION_MANHATTAN']!='out'
dir_out = check_df['COMMUTE_DIRECTION_MANHATTAN']=='out'

in_mode = check_df[dir_in].groupby(by=['PUMAKEY_HOME','MODE_TRANSP_TO_WORK_HBDMATCH']).agg({"PERWT":"sum"}).reset_index()
in_mode = in_mode.pivot_table(values='PERWT',index=['PUMAKEY_HOME'],columns='MODE_TRANSP_TO_WORK_HBDMATCH').reset_index()
in_mode.columns.name = ''

### perctange of each transmode of all commuters in the unique puma
in_mode_pct = in_mode.set_index(['PUMAKEY_HOME'])
in_mode_pct = in_mode_pct.div(in_mode_pct.sum(axis=1), axis=0).reset_index()

# in_mode[['Bus','Subway','CommuterRail','Ferry']].describe([.25,.50,.65,.75,.90])
# in_mode_pct[['Bus','Subway','CommuterRail','Ferry']].describe([.25,.50,.75,.90])

In [4]:
### set a threshold: if the percentage of bus is larger than 0.2, then this area is bus-friendly
puma_home = in_mode[['PUMAKEY_HOME']].copy(deep=True)
puma_home['Bus'] = in_mode['Bus'].apply(lambda x: 1 if x>=800 else 0)\
                    # | in_mode['PUMAKEY_HOME'].apply(lambda x: 1 if x.startswith(('36_037','36_038','36_040','36_041')) else 0)
puma_home['Subway'] = in_mode['Subway'].apply(lambda x: 1 if x>=2400 else 0)
puma_home['CommuterRail'] = in_mode['CommuterRail'].apply(lambda x: 1 if x>=1000 else 0)
puma_home['Ferry'] = in_mode['Ferry'].apply(lambda x: 1 if x>=310 else 0)
puma_home

# puma_home_Bus = puma_home[puma_home['Bus']==1]['PUMAKEY_HOME'].to_list()
# puma_home_Subway = puma_home[puma_home['Subway']==1]['PUMAKEY_HOME'].to_list()
# puma_home_CommuterRail = puma_home[puma_home['CommuterRail']==1]['PUMAKEY_HOME'].to_list()
# puma_home_Ferry = puma_home[puma_home['Ferry']==1]['PUMAKEY_HOME'].to_list()

,PUMAKEY_HOME,Bus,Subway,CommuterRail,Ferry
0,09_00100,0,0,1,0
1,09_00101,0,0,1,0
2,09_00102,0,0,1,0
3,09_00103,0,0,1,0
4,09_00104,0,0,1,0
...,...,...,...,...,...
256,44_00103,0,0,0,0
257,44_00201,0,0,0,0
258,44_00300,0,0,0,0
259,44_00400,0,0,0,0


In [5]:
out_mode = check_df[dir_out].groupby(by=['PUMAKEY_WORK','MODE_TRANSP_TO_WORK_HBDMATCH']).agg({"PERWT":"sum"}).reset_index()
out_mode = out_mode.pivot_table(values='PERWT',index=['PUMAKEY_WORK'],columns='MODE_TRANSP_TO_WORK_HBDMATCH').reset_index()
out_mode.columns.name = ''

out_mode_pct = out_mode.set_index(['PUMAKEY_WORK'])
out_mode_pct = out_mode_pct.div(out_mode_pct.sum(axis=1), axis=0).reset_index()

# out_mode[['Bus','Subway','CommuterRail','Ferry']].describe([.25,.50,.65,.75,.90])
# out_mode_pct[['Bus','Subway','CommuterRail','Ferry']].describe([.25,.50,.75,.90])

In [6]:
puma_work = out_mode[['PUMAKEY_WORK']].copy(deep=True)
puma_work['Bus'] = out_mode['Bus'].apply(lambda x: 1 if x>=0 else 0)
puma_work['Subway'] = out_mode['Subway'].apply(lambda x: 1 if x>=180 else 0)
puma_work['CommuterRail'] = out_mode['CommuterRail'].apply(lambda x: 1 if x>=0 else 0)
puma_work['Ferry'] = out_mode['Ferry'].apply(lambda x: 1 if x>=0 else 0)
puma_work

# puma_work_Bus = puma_work[puma_work['Bus']==1]['PUMAKEY_WORK'].to_list()
# puma_work_Subway = puma_work[puma_work['Subway']==1]['PUMAKEY_WORK'].to_list()
# puma_work_CommuterRail = puma_work[puma_work['CommuterRail']==1]['PUMAKEY_WORK'].to_list()
# puma_work_Ferry = puma_work[puma_work['Ferry']==1]['PUMAKEY_WORK'].to_list()

,PUMAKEY_WORK,Bus,Subway,CommuterRail,Ferry
0,06_03700,0,0,0,0
1,06_07500,1,0,1,0
2,06_08500,0,0,0,0
3,09_00100,1,1,1,0
4,09_00300,0,0,0,0
...,...,...,...,...,...
57,53_11600,0,0,0,0
58,81_00001,0,0,1,0
59,82_00001,0,0,0,0
60,83_00001,0,0,0,0


In [7]:
puma_home.to_csv("reachable_puma_home.csv",index=0)
puma_work.to_csv("reachable_puma_work.csv",index=0)

### Spatial Checking

<img src="reachable_puma.png" width="1000">

MTA Accessible Stations - tbd
https://new.mta.info/accessibility
https://new.mta.info/accessibility/stations
http://dashboards.mta.info/accessibility
https://new.mta.info/map/5346
https://comptroller.nyc.gov/reports/service-denied-accessibility-and-the-new-york-city-subway-system/
https://www.nytimes.com/interactive/2019/02/11/nyregion/nyc-subway-access.html
https://www.nycgo.com/articles/accessible-nyc-transportation/
https://www.taylormcginnis.com/housing-transportation-and-wheelchair-accessibility-in-nyc

Ticket Fare - tbd (depends on if we consider affordability)
https://new.mta.info/fares
https://www.ferry.nyc/ticketing-info/